# Javier Alejandro Mazariegos Godoy 20200223

In [ ]:
import pandas as pd

Creamos una función la cual tendrá como parámetros lo necesario para poder establecer la conexión al SQL:

In [139]:
import mysql.connector
from mysql.connector import Error

def create_connection(host_name, user_name, user_password):
    connection = None
    try:
        connection = mysql.connector.connect(
            host=host_name,
            user=user_name,
            passwd=user_password
        )
        print("Connection to MySQL DB successful")
    except Error as e:
        print(f"The error '{e}' occurred")

    return connection

In [140]:
host = "relational.fit.cvut.cz"
user = "guest"
pwd = "relational"
connection = create_connection(host_name = host, user_name =user ,user_password = pwd)

Connection to MySQL DB successful


# PARTE 1

## Problema 1

In [16]:
query = """ SELECT 
        e.first_name, 
        e.last_name, 
        e.emp_no, 
        t.title, 
        s.from_date, 
        s.to_date, 
        s.salary 
        FROM employee.employees e 
        JOIN employee.titles t on e.emp_no = t.emp_no 
        JOIN employee.salaries s on t.emp_no = s.emp_no 
        WHERE e.emp_no = 12557;
        """
employees = pd.read_sql(query,connection)
employees

,first_name,last_name,emp_no,title,from_date,to_date,salary
0,Stafford,Kopetz,12557,Senior Engineer,1988-03-20,1989-03-20,41840
1,Stafford,Kopetz,12557,Senior Engineer,1989-03-20,1990-03-20,46021
2,Stafford,Kopetz,12557,Senior Engineer,1990-03-20,1991-03-20,48699
3,Stafford,Kopetz,12557,Senior Engineer,1991-03-20,1992-03-19,51610
4,Stafford,Kopetz,12557,Senior Engineer,1992-03-19,1993-03-19,51351
5,Stafford,Kopetz,12557,Senior Engineer,1993-03-19,1994-03-19,51660
6,Stafford,Kopetz,12557,Senior Engineer,1994-03-19,1995-03-19,52878
7,Stafford,Kopetz,12557,Senior Engineer,1995-03-19,1996-03-18,54053
8,Stafford,Kopetz,12557,Senior Engineer,1996-03-18,1997-03-18,57950
9,Stafford,Kopetz,12557,Senior Engineer,1997-03-18,1998-03-18,58055


## Problema 2

In [56]:
query = """ 
        SELECT 
        e.first_name, 
        e.last_name, 
        d.emp_no,
        e.birth_date,
        dep.dept_no,
        depa.dept_name,
        dep.from_date,
        dep.to_date
        FROM 
        (SELECT 
        emp_no, 
        max(from_date) m
        FROM employee.dept_emp 
        GROUP BY 1) d
        JOIN employee.employees e on d.emp_no = e.emp_no
        JOIN employee.dept_emp dep on dep.emp_no = d.emp_no 
        JOIN employee.departments depa on dep.dept_no = depa.dept_no
        WHERE dep.from_date = d.m;
        
        
        """
employees = pd.read_sql(query,connection)
employees

,first_name,last_name,emp_no,birth_date,dept_no,dept_name,from_date,to_date
0,Mary,Sluis,10011,1953-11-07,d009,Customer Service,1990-01-22,1996-11-09
1,Huan,Lortz,10038,1960-07-20,d009,Customer Service,1989-09-20,9999-01-01
2,Basil,Tramer,10049,1961-04-24,d009,Customer Service,1992-05-04,9999-01-01
3,Breannda,Billingsley,10060,1961-10-15,d009,Customer Service,1992-11-11,9999-01-01
4,Jungsoon,Syrzycki,10088,1954-02-25,d009,Customer Service,1992-03-21,9999-01-01
...,...,...,...,...,...,...,...,...
300048,Guozhong,Felder,499976,1963-08-20,d007,Sales,1988-12-26,9999-01-01
300049,Gino,Usery,499980,1959-06-28,d007,Sales,1991-02-11,9999-01-01
300050,Nathan,Ranta,499986,1952-07-22,d007,Sales,1985-08-11,9999-01-01
300051,Rimli,Dusink,499987,1961-09-05,d007,Sales,1999-12-21,9999-01-01


## Problema 3

In [146]:
query = """
    SELECT 
    *
    FROM
        (SELECT
        emp_no,
        salary,
        from_date,
        to_date,
        row_number() OVER (PARTITION BY emp_no order by salary DESC) AS seq
        FROM employee.salaries
        group by 1,2,3,4
        order by emp_no) s
    JOIN employee.employees e ON e.emp_no = s.emp_no
    WHERE seq = 1;
"""
pd.read_sql(query,connection)

,emp_no,salary,from_date,to_date,seq,emp_no,birth_date,first_name,last_name,gender,hire_date
0,10001,88958,2002-06-22,9999-01-01,1,10001,1953-09-02,Georgi,Facello,M,1986-06-26
1,10002,72527,2001-08-02,9999-01-01,1,10002,1964-06-02,Bezalel,Simmel,F,1985-11-21
2,10003,43699,2000-12-01,2001-12-01,1,10003,1959-12-03,Parto,Bamford,M,1986-08-28
3,10004,74057,2001-11-27,9999-01-01,1,10004,1954-05-01,Chirstian,Koblick,M,1986-12-01
4,10005,94692,2001-09-09,9999-01-01,1,10005,1955-01-21,Kyoichi,Maliniak,M,1989-09-12
...,...,...,...,...,...,...,...,...,...,...,...
300019,499995,52868,2002-06-01,9999-01-01,1,499995,1958-09-24,Dekang,Lichtner,F,1993-01-12
300020,499996,69501,2002-05-12,9999-01-01,1,499996,1953-03-07,Zito,Baaz,M,1990-09-27
300021,499997,83441,2001-08-26,9999-01-01,1,499997,1961-08-03,Berhard,Lenart,M,1986-04-21
300022,499998,55003,2001-12-25,9999-01-01,1,499998,1956-09-05,Patricia,Breugel,M,1993-10-13


# PARTE 2

## Problema 1

In [153]:
import sqlite3
con = sqlite3.connect('sql-murder-mystery.db')
pd.read_sql("SELECT * FROM sqlite_master", con)

,type,name,tbl_name,rootpage,sql
0,table,crime_scene_report,crime_scene_report,2,CREATE TABLE crime_scene_report (\n dat...
1,table,drivers_license,drivers_license,3,CREATE TABLE drivers_license (\n id int...
2,table,person,person,4,CREATE TABLE person (\n id integer PRIM...
3,table,facebook_event_checkin,facebook_event_checkin,5,CREATE TABLE facebook_event_checkin (\n ...
4,table,interview,interview,6,CREATE TABLE interview (\n person_id in...
5,table,get_fit_now_member,get_fit_now_member,7,CREATE TABLE get_fit_now_member (\n id ...
6,index,sqlite_autoindex_get_fit_now_member_1,get_fit_now_member,8,None
7,table,get_fit_now_check_in,get_fit_now_check_in,9,CREATE TABLE get_fit_now_check_in (\n m...
8,table,income,income,10,CREATE TABLE income (\n ssn integer PRI...
9,table,solution,solution,11,"CREATE TABLE solution (\n user integer,..."


In [154]:
pd.read_sql("SELECT * FROM crime_scene_report", con)

,date,type,description,city
0,20180115,robbery,A Man Dressed as Spider-Man Is on a Robbery Spree,NYC
1,20180115,murder,Life? Dont talk to me about life.,Albany
2,20180115,murder,"Mama, I killed a man, put a gun against his he...",Reno
3,20180215,murder,REDACTED REDACTED REDACTED,SQL City
4,20180215,murder,Someone killed the guard! He took an arrow to ...,SQL City
...,...,...,...,...
1223,20180430,bribery,\n,Garden Grove
1224,20180430,fraud,‘Why not?’ said the March Hare.\n,Houma
1225,20180430,assault,\n,Fontana
1226,20180501,assault,be NO mistake about it: it was neither more no...,Trenton


In [186]:
resultado = pd.read_sql("""SELECT 
            *
            FROM crime_scene_report
            WHERE date = '20180115'
            AND city = 'SQL City'
            AND type = 'murder'""", con)

In [187]:
resultado["description"][0]

'Security footage shows that there were 2 witnesses. The first witness lives at the last house on "Northwestern Dr". The second witness, named Annabel, lives somewhere on "Franklin Ave".'

In [188]:
pd.read_sql("SELECT * FROM person", con)

,id,name,license_id,address_number,address_street_name,ssn
0,10000,Christoper Peteuil,993845,624,Bankhall Ave,747714076
1,10007,Kourtney Calderwood,861794,2791,Gustavus Blvd,477972044
2,10010,Muoi Cary,385336,741,Northwestern Dr,828638512
3,10016,Era Moselle,431897,1987,Wood Glade St,614621061
4,10025,Trena Hornby,550890,276,Daws Hill Way,223877684
...,...,...,...,...,...,...
10006,99936,Luba Benser,274427,680,Carnage Blvd,685095054
10007,99941,Roxana Mckimley,975942,1613,Gate St,512136801
10008,99965,Cherie Zeimantz,287627,3661,The Water Ave,362877324
10009,99982,Allen Cruse,251350,3126,N Jean Dr,348734531


### Primer testigo

In [191]:
pd.read_sql("""SELECT 
            * 
            FROM person
            WHERE address_street_name = 'Northwestern Dr' 
            ORDER BY address_number DESC
            LIMIT 1""", con)

,id,name,license_id,address_number,address_street_name,ssn
0,14887,Morty Schapiro,118009,4919,Northwestern Dr,111564949


### Segundo testigo

In [195]:
pd.read_sql("""SELECT 
            * 
            FROM person
            WHERE address_street_name = 'Franklin Ave' 
            AND name LIKE '%Annabel%' """, con)

,id,name,license_id,address_number,address_street_name,ssn
0,16371,Annabel Miller,490173,103,Franklin Ave,318771143


### Entrvista primer testigo

In [205]:
entrevista_testigo1 = pd.read_sql("""SELECT 
            * 
            FROM interview
            WHERE person_id = '14887' """, con)
entrevista_testigo1["transcript"][0]

'I heard a gunshot and then saw a man run out. He had a "Get Fit Now Gym" bag. The membership number on the bag started with "48Z". Only gold members have those bags. The man got into a car with a plate that included "H42W".'

### Entrvista segundo testigo 

In [206]:
entrevista_testigo2 = pd.read_sql("""SELECT 
            * 
            FROM interview
            WHERE person_id = '16371' """, con)
entrevista_testigo2["transcript"][0]

'I saw the murder happen, and I recognized the killer from my gym when I was working out last week on January the 9th.'

### Auto en el que escapó el presunto criminal

In [223]:
pd.read_sql("""SELECT 
                * 
                FROM drivers_license
                WHERE plate_number LIKE "%H42W%"
                AND gender = "male"
                """, con)

,id,age,height,eye_color,hair_color,gender,plate_number,car_make,car_model
0,423327,30,70,brown,brown,male,0H42W2,Chevrolet,Spark LS
1,664760,21,71,black,black,male,4H42WR,Nissan,Altima


### Membresia del presunto criminal

In [220]:
pd.read_sql("""SELECT 
                * 
                FROM get_fit_now_member
                WHERE ID LIKE "48Z%"
                AND membership_status = 'gold' """, con)

,id,person_id,name,membership_start_date,membership_status
0,48Z7A,28819,Joe Germuska,20160305,gold
1,48Z55,67318,Jeremy Bowers,20160101,gold


### Check in del presunto criminal

In [224]:
pd.read_sql("""SELECT 
                * 
                FROM get_fit_now_check_in
                WHERE check_in_date LIKE "%0109" 
                AND membership_id IN ('48Z7A','48Z55')""", con)

,membership_id,check_in_date,check_in_time,check_out_time
0,48Z7A,20180109,1600,1730
1,48Z55,20180109,1530,1700


### Verificación si el testigo 2 se encontraba en el gimnasio

In [225]:
pd.read_sql("""SELECT 
                * 
                FROM get_fit_now_member
                WHERE person_id = "16371" """, con)

,id,person_id,name,membership_start_date,membership_status
0,90081,16371,Annabel Miller,20160208,gold


In [226]:
pd.read_sql("""SELECT 
                * 
                FROM get_fit_now_check_in
                WHERE membership_id = "90081" """, con)

,membership_id,check_in_date,check_in_time,check_out_time
0,90081,20180109,1600,1700


### Información del segundo sospechoso

In [230]:
pd.read_sql("""SELECT 
                * 
                FROM facebook_event_checkin 
                WHERE person_id = '67318' """, con)

,person_id,event_id,event_name,date
0,67318,4719,The Funky Grooves Tour,20180115
1,67318,1143,SQL Symphony Concert,20171206


In [235]:
pd.read_sql("""SELECT 
                * 
                FROM income
                where ssn = '871539279' """, con)

,ssn,annual_income
0,871539279,10500


# Se concluye que el asesino fue Jeremy Bowers

In [249]:
asesino = pd.read_sql("""
                SELECT
                *
                FROM person
                WHERE ID IN
                (SELECT 
                person_id
                FROM get_fit_now_member
                WHERE ID LIKE "48Z%"
                AND membership_status = 'gold')
                AND license_id IN
                (
                SELECT 
                ID
                FROM drivers_license
                WHERE plate_number LIKE "%H42W%"
                AND gender = "male"
                )
                """, con)
asesino["name"][0]

'Jeremy Bowers'

In [245]:
cursor = con.cursor()
sql = "INSERT INTO solution VALUES (1, 'Jeremy Bowers')"
cursor.execute(sql)
con.commit()
var = pd.read_sql("SELECT value FROM solution", con)
var.iloc[0,0]


"Congrats, you found the murderer! But wait, there's more... If you think you're up for a challenge, try querying the interview transcript of the murderer to find the real villian behind this crime. If you feel especially confident in your SQL skills, try to complete this final step with no more than 2 queries. Use this same INSERT statement with your new suspect to check your answer."

### Segumineto del culpable detrás de la planificación del crimen.

In [248]:
entrevista_asesino = pd.read_sql("""SELECT 
            * 
            FROM interview
            WHERE person_id = '67318' """, con)
entrevista_asesino["transcript"][0]

'I was hired by a woman with a lot of money. I don\'t know her name but I know she\'s around 5\'5" (65") or 5\'7" (67"). She has red hair and she drives a Tesla Model S. I know that she attended the SQL Symphony Concert 3 times in December 2017.\n'

In [261]:
pd.read_sql("""SELECT 
            ID 
            FROM drivers_license
            WHERE height BETWEEN '65' AND '67'
            AND hair_color = 'red'
            AND gender = 'female'
            AND car_make = 'Tesla' 
            AND car_model = 'Model S' """, con)

,id,age,height,eye_color,hair_color,gender,plate_number,car_make,car_model
0,202298,68,66,green,red,female,500123,Tesla,Model S
1,291182,65,66,blue,red,female,08CM64,Tesla,Model S
2,918773,48,65,black,red,female,917UU3,Tesla,Model S


# La culpable detrás de la planificación del crimen fue Miranda Priestly

In [271]:
culpable_planificacion = pd.read_sql("""
            
            SELECT *
            FROM person
            WHERE id IN
                (SELECT 
                DISTINCT(person_id)
                FROM facebook_event_checkin
                WHERE event_name = 'SQL Symphony Concert'
                AND date LIKE '201712%'
                AND person_id IN 
                    (SELECT
                    ID
                    FROM person
                    WHERE license_id IN
                        (SELECT 
                        ID 
                        FROM drivers_license
                        WHERE height BETWEEN '65' AND '67'
                        AND hair_color = 'red'
                        AND gender = 'female'
                        AND car_make = 'Tesla' 
                        AND car_model = 'Model S'))) """, con)

culpable_planificacion["name"][0]

'Miranda Priestly'